<a href="https://colab.research.google.com/github/pedroblossbraga/Python-Optimization/blob/master/Python_Multiprocessamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multiprocessamento em Python**: baseando-se na documentação.

>

Referência principal: [link text](https://docs.python.org/2/library/multiprocessing.html)

https://docs.python.org/2/library/multiprocessing.html


In [1]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    p = Pool(5)
    print(p.map(f, [1, 2, 3]))


[1, 4, 9]


# **6.6.1.1. A classe Process**

In [3]:
from multiprocessing import Process

def f(name):
    print('hello', name)

if __name__ == '__main__':
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

hello bob


## Expondo os IDs dos processos:


In [7]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    if hasattr(os, 'getppid'):  # only available on Unix
        print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

main line
module name: __main__
parent process: 23
process id: 124
function f
module name: __main__
parent process: 124
process id: 210
hello bob


# **16.6.1.2. Trocando objetos entre processos**¶

o multiprocessamento suporta dois tipos de canal de comunicação entre processos:

>

 - ## **Queues**

    A classe Queue é praticamente um clone de Queue.Queue. Por exemplo:



In [9]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    print(q.get())    # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


    Queues are thread and process safe.


- ## **Pipes**
    A função Pipe () retorna um par de objetos de conexão conectados por um pipe que, por padrão, é duplex (bidirecional). Por exemplo:
    

In [11]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print(parent_conn.recv())   # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


Os dois objetos de conexão retornados por Pipe () representam as duas extremidades do pipe. Cada objeto de conexão possui os métodos send () e recv () (entre outros). 

<br>

Observe que os dados em um canal podem ficar corrompidos se dois processos (ou threads) tentarem ler ou gravar na mesma extremidade do canal ao mesmo tempo. 

<br>

Obviamente, não há risco de corrupção de processos que usam extremidades diferentes do pipe ao mesmo tempo.

# **16.6.1.3 Sincronização entre processos¶**

<br>

o multiprocessamento contém equivalentes de todas as primitivas de sincronização do encadeamento. 

<br>

Por exemplo, pode-se usar uma trava **(Lock)** para garantir que apenas um processo seja impresso na saída padrão de cada vez:








In [12]:
from multiprocessing import Process, Lock

def f(l, i):
    l.acquire()
    print('hello world', i)
    l.release()

if __name__ == '__main__':
    lock = Lock()

    for num in range(10):
        Process(target=f, args=(lock, num)).start()

hello world 0
hello world 1
hello world 2
hello world 3
hello world 4
hello world 5
hello world 6
hello world 7
hello world 8
hello world 9


Sem usar a saída de bloqueio dos diferentes processos, é possível que tudo fique confuso.

# **16.6.1.4 Estado de compartilhamento entre processos**

<br>

Como mencionado acima, ao fazer programação simultânea (concurrent), geralmente é melhor evitar o uso do estado compartilhado na medida do possível. Isso é particularmente verdade ao usar vários processos.

No entanto, se você realmente precisar usar alguns dados compartilhados, o multiprocessamento fornecerá algumas maneiras de fazer isso.

- ### **Memoria compartilhada**

Os dados podem ser armazenados em um mapa de memória compartilhada usando *Valor* ou *Array*. Por exemplo, o seguinte código:



In [14]:
from multiprocessing import Process, Value, Array

def f(n, a):
    n.value = 3.1415927
    for i in range(len(a)):
        a[i] = -a[i]

if __name__ == '__main__':
    num = Value('d', 0.0)
    arr = Array('i', range(10))

    p = Process(target=f, args=(num, arr))
    p.start()
    p.join()

    print(num.value)
    print(arr[:])

3.1415927
[0, -1, -2, -3, -4, -5, -6, -7, -8, -9]


Os argumentos 'd' e 'i' usados ​​ao criar num e arr são códigos de tipo do tipo usado pelo módulo array: 'd' indica uma flutuação de precisão dupla e 'i' indica um número inteiro assinado. Esses objetos compartilhados serão seguros para processos e threads.

<br>


Para obter mais flexibilidade no uso da memória compartilhada, pode-se usar o módulo multiprocessing.sharedctypes, que suporta a criação de objetos ctypes arbitrários alocados na memória compartilhada.



- ### **Processo do servidor**

    Um objeto gerenciador retornado por **Manager ()** controla um processo do servidor que mantém objetos Python e permite que outros processos os manipulem usando proxies.

    Um gerente retornado por **Manager ()** oferecerá suporte aos tipos *list, dict, Namespace, Lock, RLock, Semáforo, BoundedSemaphore, Condition, Event, Queue, Value e Array.* Por exemplo,



In [15]:
from multiprocessing import Process, Manager

def f(d, l):
    d[1] = '1'
    d['2'] = 2
    d[0.25] = None
    l.reverse()

if __name__ == '__main__':
    manager = Manager()

    d = manager.dict()
    l = manager.list(range(10))

    p = Process(target=f, args=(d, l))
    p.start()
    p.join()

    print(d)
    print(l)

{1: '1', '2': 2, 0.25: None}
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


Os gerenciadores de processos do servidor são mais flexíveis do que o uso de objetos de memória compartilhada porque eles podem ser criados para suportar tipos de objetos arbitrários. 

Além disso, um único gerente pode ser compartilhado por processos em computadores diferentes em uma rede. 

Eles são, no entanto, mais lentos que o uso de memória compartilhada.

# **16.6.1.5 Usando um pool de trabalhadores**

A classe *Pool* representa um conjunto de processos de trabalho. Possui métodos que permitem que as tarefas sejam transferidas para os processos de trabalho de algumas maneiras diferentes.

<br>

Por exemplo:

In [16]:
from multiprocessing import Pool, TimeoutError
import time
import os

def f(x):
    return x*x

if __name__ == '__main__':
    pool = Pool(processes=4)              # start 4 worker processes

    # print "[0, 1, 4,..., 81]"
    print(pool.map(f, range(10)))

    # print same numbers in arbitrary order
    for i in pool.imap_unordered(f, range(10)):
        print(i)

    # evaluate "f(20)" asynchronously
    res = pool.apply_async(f, (20,))      # runs in *only* one process
    print (res.get(timeout=1) )             # prints "400"

    # evaluate "os.getpid()" asynchronously
    res = pool.apply_async(os.getpid, ()) # runs in *only* one process
    print( res.get(timeout=1)  )            # prints the PID of that process

    # launching multiple evaluations asynchronously *may* use more processes
    multiple_results = [pool.apply_async(os.getpid, ()) for i in range(4)]
    print ([res.get(timeout=1) for res in multiple_results])

    # make a single worker sleep for 10 secs
    res = pool.apply_async(time.sleep, (10,))
    try:
        print( res.get(timeout=1))
    except TimeoutError:
        print ("We lacked patience and got a multiprocessing.TimeoutError")

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
0
1
4
9
16
25
36
49
64
81
400
645
[644, 644, 650, 645]
We lacked patience and got a multiprocessing.TimeoutError


Observe que os métodos de um pool só devem ser usados ​​apenas pelo processo que o criou.



